In [1]:
import requests

In [2]:
url = 'https://www.bovada.lv/services/sports/event/v2/events/A/description/football/nfl?lang=en'
#HTTPS Request - returns data
res = requests.get(url)
print(res)
#200 Status means response successful
print(res.reason)

<Response [200]>
OK


In [3]:
#Data is returned in JSON format to be interepreted
data = res.json()

In [4]:
data[0].keys()

dict_keys(['path', 'events'])

In [5]:
#This doesn't give us anything
data[0]['path']

[{'id': '241',
  'link': '/football/nfl',
  'description': 'NFL',
  'type': 'LEAGUE',
  'sportCode': 'FOOT',
  'order': 1,
  'leaf': True,
  'current': True},
 {'id': '1',
  'link': '/football',
  'description': 'Football',
  'type': 'SPORT',
  'sportCode': 'FOOT',
  'order': 6,
  'leaf': False,
  'current': False}]

In [6]:
#events looks like an array of each game
for i in data[0]['events']:
    print(i['description'])

Las Vegas Raiders @ Los Angeles Chargers
Los Angeles Rams @ Seattle Seahawks
New York Jets @ Atlanta Falcons
Detroit Lions @ Minnesota Vikings
Green Bay Packers @ Cincinnati Bengals
New Orleans Saints @ Washington Football Team
Philadelphia Eagles @ Carolina Panthers
Tennessee Titans @ Jacksonville Jaguars
Chicago Bears @ Las Vegas Raiders
Cleveland Browns @ Los Angeles Chargers
New York Giants @ Dallas Cowboys
San Francisco 49ers @ Arizona Cardinals
Buffalo Bills @ Kansas City Chiefs
Indianapolis Colts @ Baltimore Ravens


In [7]:
#Further explore the first game
game_0 = data[0]['events'][0]
game_0.keys()

dict_keys(['id', 'description', 'type', 'link', 'status', 'sport', 'startTime', 'live', 'awayTeamFirst', 'denySameGame', 'teaserAllowed', 'competitionId', 'notes', 'numMarkets', 'lastModified', 'competitors', 'displayGroups'])

In [10]:
#displayGroups looks like it has all the markets
bets = game_0['displayGroups']

In [11]:
#Differnet categories seen on bovada's website
for x in bets:
    print(x['description'])

Game Lines
Alternate Lines
Special Bets
Score Props
Receiving Props
Quarterback Props
Rushing Props
Touchdown Props
Defense/Special Teams Props
Game Props
Punt Props
Sack Props
Defensive Player Props
Field Goal Props


In [13]:
#Sample market in Receiving Props
bets[4]['markets'][0]

{'id': 'G-2W-OU.Total Receiving Yards - Austin Ekeler (LAC).119',
 'descriptionKey': 'Total Receiving Yards - Austin Ekeler (LAC)',
 'description': 'Total Receiving Yards - Austin Ekeler (LAC)',
 'key': '2W-OU',
 'marketTypeId': '-634',
 'status': 'O',
 'singleOnly': False,
 'notes': 'Must play for action',
 'period': {'id': '119',
  'description': 'Game',
  'abbreviation': 'G',
  'live': False,
  'main': True},
 'outcomes': [{'id': '922622373',
   'description': 'Over',
   'status': 'O',
   'type': 'O',
   'competitorId': '9167385-16309706',
   'price': {'id': '8761882569',
    'handicap': '38.5',
    'american': '-115',
    'decimal': '1.870',
    'fractional': '20/23',
    'malay': '0.87',
    'indonesian': '-1.15',
    'hongkong': '0.87'}},
  {'id': '922622374',
   'description': 'Under',
   'status': 'O',
   'type': 'U',
   'competitorId': '9167385-16309706',
   'price': {'id': '8761882568',
    'handicap': '38.5',
    'american': '-115',
    'decimal': '1.870',
    'fractional': 

In [14]:
#Push to dataframe for export
import pandas as pd
df = pd.DataFrame()
games = data[0]['events']
for n in range(len(games)):
    game_n = games[n]
    bets = game_n['displayGroups']
    important_markets = ['Receiving Props','Rushing Props','Quarterback Props']
    for market in important_markets:
        market_bets = [x for x in bets if x['description'] == market]
        if len(market_bets):
            markets = market_bets[0]['markets']
            for specific_market in markets:
                description = specific_market['description']
                stat = description.split('-')[0].strip()
                player = description.split('-')[1].strip()
                df.loc[player,stat] = specific_market['outcomes'][0]['price']['handicap']


    

In [15]:
df

,Total Receiving Yards,Total Receptions,Longest Reception,Total Rushing Yards,Total Rushing and Receiving Yards,Total Rushing Attempts in the game,Total Passing Yards,Total Passing Attempts in the game,Total Touchdown Passes,Total Completions,Longest Completion,Total Interceptions Thrown
Austin Ekeler (LAC),38.5,5.5,14.5,57.5,100.5,13.5,NaN,NaN,NaN,NaN,NaN,NaN
Bryan Edwards (LV),35.5,2.5,18.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Darren Waller (LV),70.5,5.5,22.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Donald Parham Jr. (LAC),6.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Henry Ruggs III (LV),49.5,3.5,21.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Hunter Renfrow (LV),48.5,4.5,19.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Jalen Guyton (LAC),19.5,NaN,12.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Jared Cook (LAC),40.5,3.5,17.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Keenan Allen (LAC),80.5,6.5,23.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mike Williams (LAC),74.5,5.5,23.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
